# Building a CNN using PyTorch

# Import Libraries

In [24]:
import pandas as pd
import numpy as np

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [27]:
import kagglehub
import os

# Import Data

In [28]:
path = kagglehub.dataset_download("zalando-research/fashionmnist")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'fashionmnist' dataset.
Path to dataset files: /kaggle/input/fashionmnist


In [29]:
df = pd.read_csv(f'{path}/fashion-mnist_train.csv')
df.shape

(60000, 785)

In [30]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Check `GPU` availability

In [31]:
device = 'cpu'
x = torch.rand(2, 3)

# Move the tensor to CUDA (if available)
if torch.cuda.is_available():
  device = torch.device("cuda")
  x_cuda = x.to(device)
  print(f"Tensor on: {x_cuda.device}") # Output: Tensor on: cuda:0

device

Tensor on: cuda:0


device(type='cuda')

# Extract Features and Labels

In [32]:
X = df.drop('label', axis=1)
X = X.values
y = df['label'].values

In [33]:
X[0][100:150]

array([136,  61,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,  88, 201, 228, 225, 255, 115,  62,
       137, 255, 235, 222, 255, 135,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,  47, 252, 234, 238, 224])

# Scale Data

In [34]:
X = X/255.0
X[0][100:150]

array([0.53333333, 0.23921569, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.34509804,
       0.78823529, 0.89411765, 0.88235294, 1.        , 0.45098039,
       0.24313725, 0.5372549 , 1.        , 0.92156863, 0.87058824,
       1.        , 0.52941176, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.18431373, 0.98823529, 0.91764706, 0.93333333, 0.87843137])

# Train Test Split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

X_train.shape, X_test.shape

((48000, 784), (12000, 784))

# `Dataset` Class

In [36]:
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.from_numpy(features).float().reshape(-1, 1, 28, 28) #-1 for placeholder
    self.labels = torch.from_numpy(labels).long()

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, index):

    return self.features[index], self.labels[index]


In [37]:
train_dataset = CustomDataset(features=X_train, labels=y_train)
test_dataset = CustomDataset(features=X_test, labels=y_test)

# DataLoader

In [38]:
train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = 64,
    shuffle= True,
    pin_memory=True
  )

In [39]:
test_loader = DataLoader(
    dataset = test_dataset,
    batch_size = 64,
    shuffle= True,
    pin_memory=True
  )

# Custom Model

In [40]:
class MyCNN(nn.Module):
  def __init__(self, input_channel=1):

    super(MyCNN, self).__init__()

    self.conv_layer = nn.Sequential(

        # First Conv Layer
        nn.Conv2d(in_channels=input_channel, out_channels=32, kernel_size=(3, 3), padding='same'),
        nn.BatchNorm2d(32),
        nn.SELU(),
        nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
        nn.AlphaDropout(p=0.1),

        # Second Conv Layer
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding='same'),
        nn.BatchNorm2d(64),
        nn.SELU(),
        nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) ,
        nn.AlphaDropout(p=0.1)
    )

    self.classifier_layer = nn.Sequential(

        # Flatten Layer
        nn.Flatten(),

        # First Neural Network
        nn.Linear(in_features=64*7*7, out_features=128),
        nn.BatchNorm1d(num_features=128),
        nn.ReLU(),
        nn.Dropout(p=0.4),

        # Second Neural Network
        nn.Linear(in_features=128, out_features=64),
        nn.BatchNorm1d(num_features=64),
        nn.ReLU(),
        nn.Dropout(p=0.2),

        # Output Neural Network
        nn.Linear(in_features=64, out_features=10)
    )

  def forward(self, features):
    model = self.conv_layer(features)
    model = self.classifier_layer(model)
    return model

## Set Learning Rate and Epochs

In [41]:
learning_rate = 0.01
epochs = 25

In [42]:
# Model Creation
model = MyCNN(1)

# model Device Changes
model = model.to(device)

# Initiate Loss Function
ce_loss = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate, weight_decay=1e-4)

## Training Pipeline

In [43]:
for epoch in range(epochs):
  losses = []

  model.train()

  for batch_features, batch_labels in  train_loader:

    # move data to gpu
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)

    # forward pass
    y_pred = model(batch_features)

    # loss calculate
    loss = ce_loss(y_pred, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update params
    optimizer.step()

    # store losses
    losses.append(loss.item())

  avg_loss = np.mean(losses)
  print(f'Epochs: {epoch+1}, Loss: {avg_loss}')

Epochs: 1, Loss: 1.004745787382126
Epochs: 2, Loss: 0.6220557247002919
Epochs: 3, Loss: 0.5393500169118245
Epochs: 4, Loss: 0.4951995965838432
Epochs: 5, Loss: 0.4642022603154182
Epochs: 6, Loss: 0.442645971874396
Epochs: 7, Loss: 0.42412140653530755
Epochs: 8, Loss: 0.4066443965037664
Epochs: 9, Loss: 0.39688077157735824
Epochs: 10, Loss: 0.38500335107247036
Epochs: 11, Loss: 0.37567240236202876
Epochs: 12, Loss: 0.3663405585885048
Epochs: 13, Loss: 0.3583480042417844
Epochs: 14, Loss: 0.35445072092612584
Epochs: 15, Loss: 0.35081634189685185
Epochs: 16, Loss: 0.3429046950340271
Epochs: 17, Loss: 0.3385026457707087
Epochs: 18, Loss: 0.33261550118525823
Epochs: 19, Loss: 0.3281441297531128
Epochs: 20, Loss: 0.3230145985484123
Epochs: 21, Loss: 0.3214988302389781
Epochs: 22, Loss: 0.314780559360981
Epochs: 23, Loss: 0.3119103982547919
Epochs: 24, Loss: 0.3056135012904803
Epochs: 25, Loss: 0.3042685801585515


# Evaluation

## Set `eval` Model (Crucial)

In [44]:
model.eval()

MyCNN(
  (conv_layer): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SELU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): AlphaDropout(p=0.1, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): SELU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (9): AlphaDropout(p=0.1, inplace=False)
  )
  (classifier_layer): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): Dropout(p=0.4, inplace=False)
    (5): Linear(in_features=128, out_features=

## Eval Code

In [45]:
total_rows = 0
correct = 0

with torch.no_grad():
  for features, labels in test_loader:

      # move data to gpu
      features = features.to(device)
      labels = labels.to(device)

      # Output from network: [batch, 10]
      y_pred = model(features)

      # Predicted class index
      _, predict = torch.max(y_pred, 1)

      # Count correct predictions
      correct += (predict == labels).sum().item()
      total_rows += labels.size(0)

  avg_accuracy = correct / total_rows
  print(f'Average Accuracy: {avg_accuracy:.4f}')


Average Accuracy: 0.9167


In [46]:
total_rows = 0
correct = 0

with torch.no_grad():
  for features, labels in train_loader:

      # move data to gpu
      features = features.to(device)
      labels = labels.to(device)

      # Output from network: [batch, 10]
      y_pred = model(features)

      # Predicted class index
      _, predict = torch.max(y_pred, 1)

      # Count correct predictions
      correct += (predict == labels).sum().item()
      total_rows += labels.size(0)

  avg_accuracy = correct / total_rows
  print(f'Average Accuracy: {avg_accuracy:.4f}')

Average Accuracy: 0.9218
